In [1]:
import gpytorch
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torchvision import transforms

gpytorch.functions.use_toeplitz = False

In [2]:
class FeatureExtractor(nn.Sequential):
    
    def __init__(self):
        super(FeatureExtractor, self).__init__(nn.Conv2d(1, 32, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(32),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2),
                                 nn.Conv2d(32, 64, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2))
        
class Bottleneck(nn.Sequential):
    
    def __init__(self):
        super(Bottleneck, self).__init__(nn.Linear(64*7*7, 128),
                                         nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128, 128),
                                 nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128,64),
                                 nn.BatchNorm1d(64))

class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.bottleneck = Bottleneck()
        self.final_layer = nn.Sequential(
                                 nn.ReLU(),
                                 nn.Linear(64,10))
    
    def forward(self, x):
        features = self.feature_extractor(x)
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        classification = self.final_layer(bottlenecked_features)
        return classification
        

In [3]:
train_mnist = torchvision.datasets.MNIST('/tmp', train=True,
                                         download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))                                              
test_mnist = torchvision.datasets.MNIST('/tmp', train=False,
                                        download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

In [4]:
train_data_loader = torch.utils.data.DataLoader(train_mnist, shuffle=True, pin_memory=True, batch_size=256)

In [5]:
criterion = nn.CrossEntropyLoss().cuda()

In [23]:
model = LeNet().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [24]:
num_epochs = 5
for i in range(num_epochs):
    for x, y in train_data_loader:
        optimizer.zero_grad()
        x = Variable(x.cuda())
        y = Variable(y.cuda())
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    print("Loss: %.3f" % loss.data[0])
    
model.eval()
test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)
avg = 0.
i = 0.
for test_batch_x, test_batch_y in test_data_loader:
    predictions = model(Variable(test_batch_x).cuda()).max(-1)[1]
    test_batch_y = Variable(test_batch_y).cuda()
    avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
    i += 1.
print('Accuracy: %.4f' % (avg / i))

('Loss: ', 0.045779529958963394)
('Loss: ', 0.024345243349671364)
('Loss: ', 0.03289826586842537)
('Loss: ', 0.05639402195811272)
('Loss: ', 0.009396865963935852)
Accuracy: 0.9913


In [25]:
list(model.bottleneck.modules())[-1].weight.data.fill_(1)
None

In [30]:
from gpytorch.kernels import RBFKernel, GridInterpolationKernel

class DeepKernel(gpytorch.Module):
    def __init__(self, model):
        super(DeepKernel, self).__init__()
        self.feature_extractor = model.feature_extractor
        self.bottleneck = model.bottleneck
        self.gp_layer = GPLayer()
        
    def forward(self, x):
        features = self.feature_extractor(x)
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        gp_output = self.gp_layer(bottlenecked_features)
        return gp_output
    
    
class LatentFunction(gpytorch.AdditiveGridInducingPointModule):
    def __init__(self):
        super(LatentFunction, self).__init__(grid_size=256, grid_bounds=[(-7, 7)],
                                             n_components=64, mixing_params=False, sum_output=False)
        cov_module = RBFKernel()
        cov_module.initialize(log_lengthscale=2)
        self.cov_module = cov_module
        
    def forward(self, x):
        mean = Variable(x.data.new(len(x)).zero_())
        covar = self.cov_module(x)
        return gpytorch.random_variables.GaussianRandomVariable(mean, covar)

    
class GPLayer(gpytorch.GPModel):
    def __init__(self, n_dims=64):
        super(GPLayer, self).__init__(gpytorch.likelihoods.SoftmaxLikelihood(n_features=64, n_classes=10))
        self.latent_function = LatentFunction()
    
    def forward(self, x):
        res = self.latent_function(x)
        return res
    

In [31]:
len(train_mnist)

60000

In [32]:
deep_kernel = DeepKernel(model).cuda()
gp_data_loader = torch.utils.data.DataLoader(train_mnist, batch_size=2048., pin_memory=True, shuffle=True)

In [33]:
# Find optimal model hyperparameters
deep_kernel.train()
optimizer = torch.optim.Adam(deep_kernel.gp_layer.parameters(), lr=0.01)
optimizer.n_iter = 0
for i in range(10):
    for j, (train_x_batch, train_y_batch) in enumerate(gp_data_loader):
        train_x_batch = Variable(train_x_batch).cuda()
        train_y_batch = Variable(train_y_batch).cuda()
        optimizer.zero_grad()
        output = deep_kernel(train_x_batch)
        loss = -deep_kernel.gp_layer.marginal_log_likelihood(output, train_y_batch, n_data=len(train_mnist))
        loss.backward()
        optimizer.n_iter += 1
        print('Iter %d/200 - Loss: %.3f' % (
            i + 1, loss.data[0],
        ))
        optimizer.step()
        
    deep_kernel.eval()
    test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

    avg = 0.
    i = 0.
    for test_batch_x, test_batch_y in test_data_loader:
        predictions = deep_kernel(Variable(test_batch_x).cuda()).argmax()
        test_batch_y = Variable(test_batch_y).cuda()
        avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
        i += 1.

    print('Score')
    print(avg / i)
    deep_kernel.train()


Iter 1/200 - Loss: 829.187
Iter 1/200 - Loss: 524.971
Iter 1/200 - Loss: 517.914
Iter 1/200 - Loss: 442.155
Iter 1/200 - Loss: 356.606
Iter 1/200 - Loss: 376.987
Iter 1/200 - Loss: 288.362
Iter 1/200 - Loss: 177.634
Iter 1/200 - Loss: 144.854
Iter 1/200 - Loss: 173.783
Iter 1/200 - Loss: 213.263
Iter 1/200 - Loss: 163.703
Iter 1/200 - Loss: 132.152
Iter 1/200 - Loss: 110.321
Iter 1/200 - Loss: 111.396
Iter 1/200 - Loss: 125.318
Iter 1/200 - Loss: 135.208
Iter 1/200 - Loss: 114.461
Iter 1/200 - Loss: 100.621
Iter 1/200 - Loss: 92.725
Iter 1/200 - Loss: 98.028
Iter 1/200 - Loss: 69.182
Iter 1/200 - Loss: 74.770
Iter 1/200 - Loss: 80.181
Iter 1/200 - Loss: 86.706
Iter 1/200 - Loss: 69.074
Iter 1/200 - Loss: 88.859
Iter 1/200 - Loss: 67.346
Iter 1/200 - Loss: 61.194
Iter 1/200 - Loss: 31.252
Score
0.9763671875
Iter 2/200 - Loss: 61.512
Iter 2/200 - Loss: 67.332
Iter 2/200 - Loss: 104.585
Iter 2/200 - Loss: 79.527
Iter 2/200 - Loss: 80.924
Iter 2/200 - Loss: 83.004
Iter 2/200 - Loss: 115.82

In [ ]:
deep_kernel.eval()
test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

avg = 0.
i = 0.
for test_batch_x, test_batch_y in test_data_loader:
    predictions = deep_kernel(Variable(test_batch_x).cuda()).probability.round()
    test_batch_y = Variable(test_batch_y.fmod(2)).cuda().float()
    avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
    i += 1.

print(avg / i)
